# Experiment 0

In [1]:
import sys
sys.path.append('/Users/kenzatazi/Documents/CDT/Code')

from load import era5, data_dir, value
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import GPy
import scipy as sp
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from mfdgp.utils.metrics import msll, r2_low_vs_high

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
import emukit
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
from emukit.multi_fidelity.convert_lists_to_array import convert_x_list_to_array, convert_xy_lists_to_arrays

## Prepare data

In [19]:
# Load data
minyear = 2000
maxyear = 2005

In [27]:
gauge_df = value.all_gauge_data(minyear, maxyear, monthly=True)
station_names = gauge_df.drop_duplicates('name')['name']

# Get CV scheme
cv_locs = np.load('/Users/kenzatazi/Documents/CDT/Code/mfdgp/experiments/exp1/cv/exp1_cv_locs.npy')
cv_locs = cv_locs.reshape(-1, 2)

station_list = []
for loc in cv_locs:
    station_row = gauge_df[(gauge_df['latitude'] == loc[1]) | (gauge_df['longitude'] == loc[0])].iloc[0]
    station_list.append(station_row['name'])
station_arr = np.array(station_list)

# Split indexes
kf = KFold(n_splits=5)

cv_train_list = []
cv_test_list = []

for train_index, test_index in kf.split(station_arr):
    [station_names.drop(station_names.loc[station_names == l].index,
                        inplace=True) for l in station_arr[test_index]]
    hf_train = station_names.values 
    hf_test = station_arr[test_index]
    cv_train_list.append(hf_train)
    cv_test_list.append(hf_test)

 /var/folders/jn/8my89c2536z23fm0t2l486340000gn/T/ipykernel_10299/3201607459.py:21: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
cv_train_list


[array(['GRAZ', 'INNSBRUCK', 'SALZBURG', 'SONNBLICK', 'WIEN', 'UCCLE',
        'ZAGREB-GRIC', 'HELSINKI-KAISANIEMI', 'JYVASKYLA-LENTOASEMA',
        'SODANKYLA', 'BOURGES', 'PARIS-14E', 'MARSEILLE-MARIGNANE',
        'HOHENPEISSENBERG', 'POTSDAM', 'ZUGSPITZE', 'CORFU', 'LARISSA',
        'METHONI', 'MILAN', 'CAGLIARI', 'ROMA-CIAMPINO',
        'VERONA-VILLAFRANCA', 'KARASJOK', 'KJOEREMSGRENDE', 'FAERDER',
        'VARDOE', 'KAUNAS', 'KLAIPEDA', 'BRAGANCA', 'LISBOA-GEOFISICA',
        'ARAD', 'BUCURESTI-BANEASA', 'BADAJOZ-TALAVERA', 'MALAGA',
        'NAVACERRADA', 'SAN-SEBASTIAN-IGUELDO',
        'TORTOSA-OBSERVATORIO-DEL-EBRO', 'BASEL-BINNINGEN', 'LUGANO',
        'SAENTIS', '\tZUERISWITZERLAND', 'ESKDALEMUIR', 'OXFORD', 'WICK',
        'RENNES', 'FOKSTUA', 'LEBA', 'SIEDLCE', 'HAPARANDA', 'STORNOWAY',
        'VALLEY', 'MONT-AIGOUAL', 'SIBIU', 'GOTEBORG', 'VISBY',
        'DRESDEN-KLOTZSCHE', 'JOKIOINEN-JOKIOISTEN', 'TOULOUSE-BLAGNAC',
        'IASI', 'BIRZAI', 'LAZDIJAI', 'SANTIAGO-D

In [29]:
# Split data
cv_x_train_hf = []
cv_y_train_hf = []
cv_x_train_lf = []
cv_y_train_lf = []
cv_x_val = []
cv_y_val = []
lf_lambdas = []


for i in range(len(cv_train_list)):

    hf_train_list = []
    for station in cv_train_list[i]:
        station_ds = value.gauge_download(
            station, minyear=minyear, maxyear=maxyear)
        hf_train_list.append(station_ds.dropna().reset_index())
    hf_train_df = pd.concat(hf_train_list)

    val_list = []
    for station in cv_test_list[i]:
        station_ds = value.gauge_download(
            station, minyear=minyear, maxyear=maxyear)
        val_list.append(station_ds.dropna().reset_index())
    val_df = pd.concat(val_list)

    era5_df = era5.value_gauge_download(
        list(cv_test_list[i]) + list(cv_train_list[i]), minyear=minyear, maxyear=maxyear)

    lf_train_df = era5_df.reset_index()
    
    # Prepare data
    
    # Transformations
    lf_train_df['tp_tr'], lf_lambda = sp.stats.boxcox(
        lf_train_df['tp'].values + 0.01)
    hf_train_df['tp_tr'] = sp.stats.boxcox(
        hf_train_df['tp'].values + 0.01, lmbda=lf_lambda)
    val_df['tp_tr'] = sp.stats.boxcox(
        val_df['tp'].values + 0.01, lmbda=lf_lambda)

    # Splitting
    x_train_lf = lf_train_df[['time', 'lat', 'lon', 'z']].values.reshape(-1, 4)
    y_train_lf = lf_train_df['tp_tr'].values.reshape(-1, 1)
    x_train_hf = hf_train_df[['time', 'latitude', 'longitude', 'altitude']].values.reshape(-1, 4)
    y_train_hf = hf_train_df[['tp_tr']].values.reshape(-1, 1)
    x_val = val_df[['time', 'latitude', 'longitude', 'altitude']].values.reshape(-1, 4)
    y_val = val_df['tp_tr'].values.reshape(-1, 1)

    # Scaling
    scaler = StandardScaler().fit(x_train_hf)
    x_train_hf1 = scaler.transform(x_train_hf)
    x_train_lf1 = scaler.transform(x_train_lf)
    x_val1 = scaler.transform(x_val)
    
    cv_x_train_hf.append(x_train_hf1)
    cv_y_train_hf.append(y_train_hf)
    cv_x_train_lf.append(x_train_lf1)
    cv_y_train_lf.append(y_train_lf)
    cv_x_val.append(x_val1)
    cv_y_val.append(y_val)
    lf_lambdas.append(lf_lambda)


value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv
value
/Users/kenzatazi/Documents/CDT/Code/data/ERA5/combi_data_value_03-2023.csv


In [31]:
np.save('data/cv_x_train_hf_value0_2000-2005.npy', cv_x_train_hf)
np.save('data/cv_y_train_hf_value0_2000-2005.npy', cv_y_train_hf)
np.save('data/cv_x_train_lf_value0_2000-2005.npy', cv_x_train_lf)
np.save('data/cv_y_train_lf_value0_2000-2005.npy', cv_y_train_lf)
np.save('data/cv_y_val_value0_2000-2005.npy', cv_y_val)
np.save('data/cv_x_val_value0_2000-2005.npy', cv_x_val)
np.save('data/lf_lambda0_2000-2005.npy', lf_lambdas)

 /Users/kenzatazi/miniconda3/envs/mfdgp/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning:Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


## Load data

In [32]:
cv_x_train_hf = np.load('data/cv_x_train_hf_value0_2000-2005.npy', allow_pickle=True)
cv_y_train_hf = np.load('data/cv_y_train_hf_value0_2000-2005.npy', allow_pickle=True)
cv_x_train_lf = np.load('data/cv_x_train_lf_value0_2000-2005.npy', allow_pickle=True)
cv_y_train_lf = np.load('data/cv_y_train_lf_value0_2000-2005.npy', allow_pickle=True)
cv_x_val = np.load('data/cv_x_val_value0_2000-2005.npy', allow_pickle=True)
cv_y_val = np.load('data/cv_y_val_value0_2000-2005.npy', allow_pickle=True)
lf_lambdas = np.load('data/lf_lambda0_2000-2005.npy', allow_pickle=True)

## MFDGP

In [9]:
R2_all = []
RMSE_all = []
RMSE_p5 = []
RMSE_p95 = []
MSLL = []

R2_all_low = []
RMSE_all_low = []
RMSE_p5_low = []
RMSE_p95_low = []
MSLL_low = []

r2_high_list = []
r2_low_list = []

for i in range(len(cv_x_train_hf)):

    # Input data
    X_train, Y_train = convert_xy_lists_to_arrays([cv_x_train_lf[i], cv_x_train_hf[i]], [cv_y_train_lf[i], cv_y_train_hf[i]])
    
    # Train and evaluate
    kern1 = GPy.kern.Matern52(input_dim=4, ARD=True)
    kernels = [kern1, GPy.kern.Matern52(input_dim=4, ARD=True)]
    lin_mf_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(kernels)
    gpy_lin_mf_model = GPyLinearMultiFidelityModel(X_train, Y_train, lin_mf_kernel, n_fidelities=2,)
    gpy_lin_mf_model.mixed_noise.Gaussian_noise.fix(0)
    gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0)
    gpy_lin_mf_model.multifidelity.Mat52_1.lengthscale[[2]] = 0.5
    lin_mf_model = GPyMultiOutputWrapper(gpy_lin_mf_model, 2, n_optimization_restarts=5)
    lin_mf_model.optimize()
    print(gpy_lin_mf_model.multifidelity.Mat52_1.lengthscale)

    # Load and prep test data                                     
    x_val1, y_val = cv_x_val[i], cv_y_val[i]                                                                                   
    n = x_val1.shape[0]
    x_met = convert_x_list_to_array([x_val1, x_val1])
    y_pred0, y_var0 = lin_mf_model.predict(x_met[n:])
    y_pred_low0, y_var_low0 = lin_mf_model.predict(x_met[:n])
    
    # ALL
    y_pred = sp.special.inv_boxcox(y_pred0, lf_lambdas[i]).reshape(-1)
    y_true = sp.special.inv_boxcox(y_val, lf_lambdas[i]).reshape(-1)
    R2_all.append(r2_score(y_true, y_pred))
    RMSE_all.append(mean_squared_error(y_true, y_pred, squared=False))
    
    y_pred_low = sp.special.inv_boxcox(y_pred_low0, lf_lambda).reshape(-1)
    R2_all_low.append(r2_score(y_true, y_pred_low))
    RMSE_all_low.append(mean_squared_error(y_true, y_pred_low, squared=False))

    # 5th PERCENTILE
    p5 = np.percentile(y_true, 5.0)
    indx = [y_true <= p5][0]
    x_val_p5 = x_val1[indx, :]
    y_true_p5 = y_true[indx]
    y_pred_p5 = y_pred[indx]
    y_pred_p5_low = y_pred_low[indx]
    RMSE_p5.append(mean_squared_error(y_true_p5, y_pred_p5, squared=False))
    RMSE_p5_low.append(mean_squared_error(y_true_p5, y_pred_p5_low, squared=False))

    # 95th PERCENTILE
    p95 = np.percentile(y_true, 95.0)
    indx = [y_true >= p95][0]
    x_val_p95 = x_val1[indx]
    y_true_p95 = y_true[indx]
    y_pred_p95 = y_pred[indx]
    y_pred_p95_low = y_pred_low[indx]
    RMSE_p95.append(mean_squared_error(y_true_p95, y_pred_p95, squared=False))
    RMSE_p95_low.append(mean_squared_error(y_true_p95, y_pred_p95_low, squared=False))
                        
    # MSLL
    ll = msll(y_val, y_pred0, y_var0)
    ll_low = msll(y_val, y_pred_low0, y_var_low0)
    MSLL.append(ll)
    MSLL_low.append(ll_low)
    
    r2_high, r2_low = r2_low_vs_high(y_pred_low, y_pred, x_val1, y_true)
    r2_high_list.append(r2_high)
    r2_low_list.append(r2_low)

 /Users/kenzatazi/miniconda3/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:divide by zero encountered in log


In [30]:
print(gpy_lin_mf_model.multifidelity.Mat52_1.lengthscale)


  index  |  gp.multifidelity.Mat52_1.lengthscale  |  constraints  |  priors
  [0]    |                            0.03993688  |      +ve      |        
  [1]    |                            0.20328902  |      +ve      |        
  [2]    |                            0.12667011  |      +ve      |        
  [3]    |                            0.48907254  |      +ve      |        


In [36]:
print('Mean RMSE = ', np.mean(RMSE_all), '±', np.std(RMSE_all))
print('Mean R2 = ', np.mean(R2_all), '±', np.std(R2_all))
print('5th RMSE = ', np.mean(RMSE_p5), '±', np.std(RMSE_p5))
print('95th RMSE = ', np.mean(RMSE_p95), '±', np.std(RMSE_p95))
print('MSLL= ', np.mean(MSLL), '±', np.std(MSLL))
                        
print('Mean RMSE = ', np.mean(RMSE_all_low), '±', np.std(RMSE_all_low))
print('Mean R2 = ', np.mean(R2_all_low), '±', np.std(R2_all_low))
print('5th RMSE = ', np.mean(RMSE_p5_low), '±', np.std(RMSE_p5_low))
print('95th RMSE = ', np.mean(RMSE_p95_low), '±', np.std(RMSE_p95_low))
print('MSLL= ', np.mean(MSLL_low), '±', np.std(MSLL_low))

np.save('exp1_ypred_lf_r2_2000-2005.npy', r2_low_list)
np.save('exp1_ypred_hf_r2_2000-2005.npy', r2_high_list)

Mean RMSE =  1.1288174216277413 ± 0.45955825345996737
Mean R2 =  0.6180231256311904 ± 0.10646565424263782
5th RMSE =  0.5652902009825029 ± 0.23832664507962512
95th RMSE =  3.001644981397689 ± 1.5755066408650757
MSLL=  0.9044514227576208 ± 0.19855495122324227
Mean RMSE =  1.2059885816345628 ± 0.4592714269204212
Mean R2 =  0.5474398303518873 ± 0.14323051517276358
5th RMSE =  0.5875472687144019 ± 0.29444462409967453
95th RMSE =  2.841226967200339 ± 1.1729332246925321
MSLL=  18705811.2223 ± 7429526.352754536


In [33]:
R2_all

[0.6185843494697647,
 0.43893394697099797,
 0.6464235774513374,
 0.7768774566962863,
 0.6166973725289622]

## GP model on gauge data

In [13]:
R2_all = []
RMSE_all = []
RMSE_p5 = []
RMSE_p95 = []
MSLL = []

for i in range(len(cv_train_list)):

    x_train_hf1, y_train_hf = cv_x_train_hf[i], cv_y_train_hf[i]
    x_val1, y_val = cv_x_val[i], cv_y_val[i]
    
    # Input data
    kernel = GPy.kern.StdPeriodic(1, active_dims=[0], period=1) * GPy.kern.RBF(1, active_dims=[0]) + GPy.kern.RBF(3, active_dims=[1,2,3], ARD=True)
    m = GPy.models.GPRegression(x_train_hf1, y_train_hf, kernel)
    m.optimize_restarts(num_restarts = 5)
    
    y_pred0, y_var0 = m.predict(x_val1)
    
    y_pred = sp.special.inv_boxcox(y_pred0, lf_lambda).reshape(-1)
    y_true = sp.special.inv_boxcox(y_val, lf_lambda).reshape(-1)
    R2_all.append(r2_score(y_true, y_pred))
    RMSE_all.append(mean_squared_error(y_true, y_pred, squared=False))
    
    # 5th PERCENTILE
    p5 = np.percentile(y_true, 5.0)
    indx = [y_true <= p5][0]
    x_val_p5 = x_val1[indx, :]
    y_true_p5 = y_true[indx]
    y_pred_p5 = y_pred[indx]
    RMSE_p5.append(mean_squared_error(y_true_p5, y_pred_p5, squared=False))

    # 95th PERCENTILE
    p95 = np.percentile(y_true, 95.0)
    indx = [y_true >= p95][0]
    x_val_p95 = x_va1l[indx]
    y_true_p95 = y_true[indx]
    y_pred_p95 = y_pred[indx]
    RMSE_p95.append(mean_squared_error(y_true_p95, y_pred_p95, squared=False))
    
    # MSLL
    ll = msll(y_val, y_pred0, y_var0)
    MSLL.append(ll)

  index  |  gp.multifidelity.rbf.lengthscale  |  constraints  |  priors
  [0]    |                        0.04363753  |      +ve      |        
  [1]    |                        0.33917221  |      +ve      |        
  [2]    |                        0.26411480  |      +ve      |        
  [3]    |                        0.49906948  |      +ve      |        


In [27]:
print('GP on gauges')
print('Mean RMSE = ', np.mean(RMSE_all), '±', np.std(RMSE_all))
print('Mean R2 = ', np.mean(R2_all), '±', np.std(R2_all))
print('5th RMSE = ', np.mean(RMSE_p5), '±', np.std(RMSE_p5))
print('95th RMSE = ', np.mean(RMSE_p95), '±', np.std(RMSE_p95))
print('MSLL= ', np.mean(MSLL), '±', np.std(MSLL))

GP on gauges
Mean RMSE =  1.766620778217169 ± 0.45693719979232983
Mean R2 =  -0.018762063464400326 ± 0.12908989365533002
5th RMSE =  1.8763025060321297 ± 0.2470065166182675
95th RMSE =  5.1869952472261325 ± 1.7614928997790533
MSLL=  nan ± nan


 /Users/kenzatazi/miniconda3/envs/mfdgp/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning:Mean of empty slice.
 /Users/kenzatazi/miniconda3/envs/mfdgp/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning:invalid value encountered in double_scalars
 /Users/kenzatazi/miniconda3/envs/mfdgp/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning:Degrees of freedom <= 0 for slice
 /Users/kenzatazi/miniconda3/envs/mfdgp/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning:invalid value encountered in divide
 /Users/kenzatazi/miniconda3/envs/mfdgp/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning:invalid value encountered in double_scalars


## Linear regression on ERA5 data

In [20]:
R2_all = []
RMSE_all = []
RMSE_p5 = []
RMSE_p95 = []
MSLL = []

for i in range(len(cv_train_list)):

    x_train_lf1, y_train_lf = cv_x_train_lf[i], cv_y_train_lf[i]
    x_val1, y_val = cv_x_val[i], cv_y_val[i]
    
    # Train and evaluate
    linear_m = LinearRegression()
    linear_m.fit(x_train_lf1, y_train_lf)
    y_pred0 = linear_m.predict(x_val1)
   
    # ALL
    y_pred = sp.special.inv_boxcox(y_pred0, lf_lambda).reshape(-1)
    y_true = sp.special.inv_boxcox(y_val, lf_lambda).reshape(-1)
    R2_all.append(r2_score(y_true, y_pred))
    RMSE_all.append(mean_squared_error(y_true, y_pred, squared=False))
    
    # 5th PERCENTILE
    p5 = np.percentile(y_true, 5.0)
    indx = [y_true <= p5][0]
    x_val_p5 = x_val1[indx, :]
    y_true_p5 = y_true[indx]
    y_pred_p5 = y_pred[indx]
    RMSE_p5.append(mean_squared_error(y_true_p5, y_pred_p5, squared=False))
   
    # 95th PERCENTILE
    p95 = np.percentile(y_true, 95.0)
    indx = [y_true >= p95][0]
    x_val_p95 = x_val1[indx]
    y_true_p95 = y_true[indx]
    y_pred_p95 = y_pred[indx]
    RMSE_p95.append(mean_squared_error(y_true_p95, y_pred_p95, squared=False))

In [22]:
print('Lin Reg on ERA5')
print('Mean RMSE = ', np.mean(RMSE_all), '±', np.std(RMSE_all))
print('Mean R2 = ', np.mean(R2_all), '±', np.std(R2_all))
print('5th RMSE = ', np.mean(RMSE_p5), '±', np.std(RMSE_p5))
print('95th RMSE = ', np.mean(RMSE_p95), '±', np.std(RMSE_p95))

Lin Reg on ERA5
Mean RMSE =  1.766620778217169 ± 0.45693719979232983
Mean R2 =  -0.018762063464400326 ± 0.12908989365533002
5th RMSE =  1.8763025060321297 ± 0.2470065166182675
95th RMSE =  5.1869952472261325 ± 1.7614928997790533
